# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from requests.exceptions import MissingSchema
import json

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
# need to install citipy [pip install citipy via terminal]
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [27]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
# cities

600

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [30]:
# base url:
units = 'imperial'
base_url = f"api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={units}&q="

# query_url = f"base_url {city} &appid= {weather_api_key}"

# create lists for dataFrame
city_name = []
city_lat = []
city_lang = []
city_cloud = []
city_temp = []
city_maxtemp = []
city_humidity = []
city_wind_speed = []
city_country = []
date = []
# set couter for city and set
item = 1
batch = 1

# begin to display data
print('Beginning data retrival...')
# loop thourgh list of citys to request data for each city
for city in cities:
    query_url = base_url + city
    response = requests.get(query_url).json()
    try:
        print(f"Processing record {item} of set {batch} | {city}")
        city_name.append(response['name'])
        city_lat.append(response['coord']['lat'])
        city_lang.append(response['coord']['lon'])
        city_cloud.append(response['clouds']['all'])
        city_temp.append(response['main']['temp'])
        city_maxtemp.append(response['main']['temp_max'])
        city_humidity.append(response['main']['humidity'])
        city_wind_speed.append(response['wind']['speed'])
        date.append(response['dt'])
        
        item += 1
        # limit # of api calls
        if item > 50:
              batch += 1
              # reset item
              item = 1
              time.sleep(2)
    except:
        print(f"City {city} not found. Skipping...")


Beginning data retrival...


MissingSchema: Invalid URL 'api.openweathermap.org/data/2.5/weather?appid=6cdacdddfba572011f9a399c3e340102&units=imperial&q=aklavik': No schema supplied. Perhaps you meant http://api.openweathermap.org/data/2.5/weather?appid=6cdacdddfba572011f9a399c3e340102&units=imperial&q=aklavik?

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [34]:
# create df from retriaved data
city_df = pd.DataFrame({'City':city_name,
                        'Country':city_country,
                        'Date':date,
                        'Latitude':city_lat,
                        'Longitude':city_lang,
                        'Cloudiness':city_cloud,
                        'Max Temp (f)':city_maxtemp,
                        'Humidity (%)':city_humidity,
                        'Wind speed (mph)':city_wind_speed
                        })

# convert times using pd.to_datetime
city_df['Date'] = pd.to_datetime(city_df['Date'], unit = 's')
# save df to csv
city_df.to_csv('../output_data/cities.csv', index=True, index_label = 'City ID', header=True)
city_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness,Max Temp (f),Humidity (%),Wind speed (mph)


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# find cities with humidity > 100%
humidity100 = city_df.loc[city_df['Humidity (%)']>100, :]

    

In [ ]:
#  Get the indices of cities that have humidity over 100%.

if humidity100.empty==True:
    print(f"There are not cities with humidity > 100 \n Using original DataFrame to plot...")
    latidudes = city_df['Lat']
    temperatures = city_df['Max Temp (f)']
    humidity = city_df['Humidity (%)']
    cloudiness = city_df['Cloudiness (%)']
    wind_speed = city_df['Wind Speed (mph)']
else:
    humidity100ix = humidity100.index
    clean_city_df = city_df.drop(humidity10iix).reset_index(drop=True, inplace=False)
    
    latidudes = clean_city_df['Lat']
    temperatures = clean_city_df['Max Temp (f)']
    humidity = clean_city_df['Humidity (%)']
    cloudiness = clean_city_df['Cloudiness (%)']
    wind_speed = clean_city_df['Wind Speed (mph)']
    
  

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
  # export the new df to csv
    clean_city_df.to_csv(../output_data/cleancities.csv, index=True, index_lable='City ID', header=True)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression